In [253]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
import os
import PIL
from PIL import Image
from torch.utils.data import Dataset , Subset , DataLoader
import torchvision.models as models
import torch.optim as optim
import random
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter

In [254]:
transform = transforms.Compose([
    transforms.Resize((256 , 256)) , 
    transforms.PILToTensor() , 
    transforms.ConvertImageDtype(torch.float32)
    ])

In [255]:
data_dir = "/home/aman/code/CV/throat_infection/data"

classes = sorted(os.listdir(data_dir))

class_to_label = {class_name: label for label ,class_name in enumerate(classes)}




In [256]:
class CustomImageDataset(Dataset):

    def __init__(self , data_dir , transforms=None):
        self.data_dir = data_dir
        self.transform = transforms
        self.image_paths = []
        self.labels = []

        for class_name , label in class_to_label.items():
            class_dir = os.path.join(data_dir , class_name)

            for image_path in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir , image_path))
                self.labels.append(label)
        
        random.shuffle(self.image_paths)

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self ,idx):
        random.shuffle(self.image_paths)
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)
        label = self.labels[idx]
        return image, label

                

In [257]:
dataset = CustomImageDataset(data_dir=data_dir , transforms=transform)


In [258]:

def get_random_subset(ds , num_sample):
    all_indices = list(range(len(dataset)))

    random.shuffle(all_indices)
    random.shuffle(all_indices)
    random.shuffle(all_indices)

    train_indices = all_indices[:num_sample]
    valid_indices = all_indices[num_sample:]

    train_sub = Subset(ds , train_indices)
    valid_sub = Subset(ds , valid_indices)


    return train_sub , valid_sub
 

In [259]:
train_dataset , valid_dataset = get_random_subset(dataset ,4634)

device = torch.device("cuda")
batch_size = 32
train_dl = DataLoader(train_dataset , batch_size=batch_size , shuffle=True , pin_memory=True)
valid_dl = DataLoader(valid_dataset , batch_size=batch_size , shuffle=False , pin_memory=True)\




In [260]:
device = torch.device("cuda")

In [261]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(8 * 8 * 128, 256) 
        self.fc2 = nn.Linear(256 , 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(1, 8 * 8 * 128)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        print(x.shape)
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze(1)
        # return x
    

model= Net().to(device)


In [262]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [263]:
def train(model, n_epoch, train_dl, valid_dl, use_cuda=True):
    device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")
    loss_hist_train = [0] * n_epoch
    loss_hist_valid = [0] * n_epoch
    acc_hist_valid = [0] * n_epoch
    acc_hist_train = [0] * n_epoch

    model.to(device)

    for epoch in range(n_epoch):
        model.train()
        for x, y in train_dl:
            x, y = x.to(device), y.to(device).float()  # Convert target labels to float
            pred = model(x).squeeze(1)  # Remove the extra dimension from the output
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item() * y.size(0)
            is_correct = (torch.round(torch.sigmoid(pred)) == y).float()  # Apply sigmoid and round
            acc_hist_train[epoch] += is_correct.sum()
            # print("Logging training loss")
            # writer.add_scalar("Loss/train", loss.item(), epoch)  # Log training loss
            # writer.add_scalar("Accuracy/train", is_correct.sum(), epoch)  # Log training accuracy
        loss_hist_train[epoch] /= len(train_dl.dataset)
        acc_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
            for x, y in valid_dl:
                x, y = x.to(device), y.to(device).float()  # Convert target labels to float
                pred = model(x).squeeze(1)  # Remove the extra dimension from the output
                loss = loss_fn(pred, y)
                loss_hist_valid[epoch] += loss.item() * y.size(0)
                is_correct = (torch.round(torch.sigmoid(pred)) == y).float()  # Apply sigmoid and round
                acc_hist_valid[epoch] += is_correct.sum()
            loss_hist_valid[epoch] /= len(valid_dl.dataset)
            acc_hist_valid[epoch] /= len(valid_dl.dataset)
        print(f"epoch {epoch+1} accuracy {acc_hist_train[epoch]:.4f} val_accuracy: {acc_hist_valid[epoch]:.4f}")

    return loss_hist_train, loss_hist_valid, acc_hist_train, acc_hist_valid

In [264]:
torch.manual_seed(1)

num_epoch = 15

hist = train(model , num_epoch , train_dl , valid_dl)



RuntimeError: shape '[1, 8192]' is invalid for input of size 4194304